# Profile Ad-Word Datasets

In [1]:
# Copyright 2025 Luke Moffett
# Licensed under the Apache License, Version 2.0

import pandas as pd
import matplotlib.pyplot as plt
import torch

from clz_or_cls import datasets as corc_ds

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

plt.rcParams['figure.dpi'] = 300
%matplotlib inline

In [2]:
train_df, test_df, valid_df = [corc_ds.generated_df('visual+phonetic+typo_full', split) for split in ['train', 'test', 'valid']]

In [3]:
train_df['split'] = 'train'
test_df['split'] = 'test'
valid_df['split'] = 'valid'

In [ ]:
[df[['clean']].drop_duplicates('clean').nunique() for df in [train_df, test_df, valid_df]]

In [ ]:
train_df.shape

In [ ]:
test_df.shape

In [ ]:
valid_df.shape

In [ ]:
train_df.drop_duplicates(['clean', 'perturbed']).shape

In [ ]:
df = pd.concat([train_df, test_df, valid_df])
df.shape

In [ ]:
df.drop_duplicates(['clean', 'perturbed']).shape

In [11]:
316661/392646

0.8064796279600455

In [ ]:
df.drop_duplicates('perturbed').shape

In [14]:
datasets = [
    ("repeated", None),
    ("legit_extended", "visual"),
    ("dces", "visual"),
    ("ices", "visual"),
    ("zeroe_noise", "typo"),
    ("zeroe_typo", "typo"),
    ("anthro_typo", "typo"),
    ("anthro_phonetic", "phonetic"),
    ("phonee", "phonetic"),
    ("zeroe_phonetic", "phonetic")
]

class_map = {t[0]: t[1] for t in datasets}

df = None

for dataset, clazz in datasets:
    for split in ['train', 'test', 'valid']:
        ds = corc_ds.generated_df(dataset, split=split)
        ds['source'] = dataset
        ds['class'] = clazz
        ds['split'] = split

        if df is None:
            df = ds
        else:
            df = pd.concat([df, ds])

df.head(3)

,clean,perturbed,legible,legibility_score,source,class,split
0,distributed,distributed,True,1.0,repeated,None,train
1,distributed,distributed,True,1.0,repeated,None,train
2,specified,specified,True,1.0,repeated,None,train


In [15]:
df_uniq = df.drop_duplicates(['clean', 'perturbed', 'source'])
df_unique_groups = df_uniq.groupby(['source', 'split']).nunique()

df_unique_groups

clean  perturbed  legible  legibility_score  class
source          split                                                    
anthro_phonetic test      96        170        0                 0      1
                train    100        166        0                 0      1
                valid    100        171        0                 0      1
anthro_typo     test      96        157        0                 0      1
                train    100        163        0                 0      1
                valid    100        162        0                 0      1
dces            test      96        200        0                 0      1
                train    100        199        0                 0      1
                valid    100        200        0                 0      1
ices            test      96        200        0                 0      1
                train    100        200        0                 0      1
                valid    100        200        0                 0      1
legit_extended  test      96        194        2                 0      1
                train    100        193        2                 0      1
                valid    100        197        2                 0      1
phonee          test      96        187        0                 0      1
                train    100        192        0                 0      1
                valid    100        190        0                 0      1
repeated        test      96         96        1                 1      0
                train    100        100        1                 1      0
                valid    100        100        1                 1      0
zeroe_noise     test      96        194        0                 0      1
                train    100        194        0                 0      1
                valid    100        198        0                 0      1
zeroe_phonetic  test      96        187        0                 0      1
                train    100        192        0                 0      1
                valid    100        190        0                 0      1
zeroe_typo      test      96        174        0                 0      1
                train    100        171        0                 0      1
                valid    100        169        0                 0      1

In [16]:
df_unique_groups.reset_index().pivot(index=['split'], columns=['source'], values=['perturbed']).transpose()

split                      test  train  valid
          source                             
perturbed anthro_phonetic   170    166    171
          anthro_typo       157    163    162
          dces              200    199    200
          ices              200    200    200
          legit_extended    194    193    197
          phonee            187    192    190
          repeated           96    100    100
          zeroe_noise       194    194    198
          zeroe_phonetic    187    192    190
          zeroe_typo        174    171    169

In [17]:
__df = df_unique_groups.reset_index()
print('unique')
__df[__df['source'] == 'repeated'][['source', 'split', 'clean']].transpose()

unique


,18,19,20
source,repeated,repeated,repeated
split,test,train,valid
clean,96,100,100


In [18]:
__df = df.groupby(['source', 'split']).count().reset_index()
print('total')
__df[__df['source'] == 'repeated'][['source', 'split', 'clean']].transpose()

total


,18,19,20
source,repeated,repeated,repeated
split,test,train,valid
clean,200,200,200


In [ ]:
visual_ds = corc_ds.visual()
visual_ctx_ds = corc_ds.visual_ctx()
phonetic_ctx_ds = corc_ds.phonetic_ctx()
typo_ctx_ds = corc_ds.typo_ctx()

In [ ]:
visual_ds['train']['clean'][:5]

['distributed', 'distributed', 'specified', 'specified', 'exec']

In [ ]:
visual_ctx_ds['train']['clean'][:5], visual_ctx_ds['train']['perturbed'][:5]

(['distributed', 'distributed', 'specified', 'specified', 'exec'],
 ['method: "visual", word: "dİstriduțed"',
  'method: "visual", word: "distribu˵ed"',
  'method: "visual", word: "зpeσifiɘd"',
  'method: "visual", word: "ӭpecıքied"',
  'method: "visual", word: "exϲc"'])

In [ ]:
phonetic_ctx_ds['train']['perturbed'][:5], typo_ctx_ds['train']['perturbed'][:5]

(['method: "phonetic", word: "distteriebiuted"',
  'method: "phonetic", word: "destribiutted"',
  'method: "phonetic", word: "speacifid"',
  'method: "phonetic", word: "spesifid"',
  'method: "phonetic", word: "ehxec"'],
 ['method: "typo", word: "disributed"',
  'method: "typo", word: "dstbtiierud"',
  'method: "typo", word: "sicefpied"',
  'method: "typo", word: "sefceiipd"',
  'method: "typo", word: "%e$xe<c"'])

In [ ]:
vis_typ = corc_ds.geneterated_ds_ctx('visual+typo', splits=['train', 'test', 'valid'])
vis_typ

In [ ]:
vis = corc_ds.geneterated_ds_ctx('visual', splits=['train', 'test', 'valid'], ctx={'visual': 'Strategy A', 'phonetic': 'Strategy B', 'typo': 'Strategy C'})

In [23]:
vis['train']['perturbed']

['method: "Strategy A", word: "ɑiɜtrΊbuҨed"',
 'method: "Strategy A", word: "ôìsthiéՇteԀ"',
 'method: "Strategy A", word: "speƃƚfiʙd"',
 'method: "Strategy A", word: "ʚpeciͳӏeɹ"',
 'method: "Strategy A", word: "exez"',
 'method: "Strategy A", word: "сxec"',
 'method: "Strategy A", word: "poɞsesз"',
 'method: "Strategy A", word: "ρбƃsesɛ"',
 'method: "Strategy A", word: "suıtɛ"',
 'method: "Strategy A", word: "suìfs"',
 'method: "Strategy A", word: "mɛcιԀɾa"',
 'method: "Strategy A", word: "montanƌ"',
 'method: "Strategy A", word: "Ǌetro"',
 'method: "Strategy A", word: "metɦo"',
 'method: "Strategy A", word: "ɿarker"',
 'method: "Strategy A", word: "parƙer"',
 'method: "Strategy A", word: "ċhemicaɨs"',
 'method: "Strategy A", word: "zheǌiŏals"',
 'method: "Strategy A", word: "sɛientɫfĭc"',
 'method: "Strategy A", word: "сcienƭific"',
 'method: "Strategy A", word: "Ԁeϧieо"',
 'method: "Strategy A", word: "ɑeȧIϵü"',
 'method: "Strategy A", word: "pɛlӌ"',
 'method: "Strategy A", word: "pò

In [ ]:
full = corc_ds.geneterated_ds_ctx('visual+phonetic+typo_full', splits=['train', 'test', 'valid'], ctx={'visual': 'Strategy A', 'phonetic': 'Strategy B', 'typo': 'Strategy C'})

In [25]:
full['train']['perturbed'][-6:]

['method: "Strategy C", word: "di$s^close"',
 'method: "Strategy C", word: "dssolcie"',
 'method: "Strategy C", word: "surv`ivo<r"',
 'method: "Strategy C", word: "_sur<vivo:r"',
 'method: "Strategy C", word: "k,ids"',
 'method: "Strategy C", word: "kis"']

In [ ]:
corc_ds.geneterated_ds_ctx('visual+typo+phonetic_full', splits=['train', 'test', 'valid'])